In [7]:
import pandas, numpy

In [8]:
file_directory = "/Users/kja11/OneDrive - Menntaský/PostDoc_Hypothermia/in_silico/Python/"

# Define Paths

In [70]:
# proteomic data
proteomics_path = file_directory+'3) output/Proteomics/'

# RNAseq data
RNAseq_path = file_directory+'3) output/RNAseq/'
RNAseq_path2 = file_directory+'1) input/RNAseq/'

# GeCKO data
GeCKO_path = file_directory+'1) input/GeCKO/'

# Open the files

In [65]:
#open .txt
with open(proteomics_path+'signif_linregress_proteins.txt') as f:
    proteomics_pval = f.read().splitlines()

with open(proteomics_path+'adjPvalue_linregress_proteins.txt') as f:
    proteomics_adjpval = f.read().splitlines() 
    
with open(proteomics_path+'high_Rval_prot_linregress_proteins.txt') as f:
    proteomics_high_rval = f.read().splitlines() 
    
with open(RNAseq_path+'allsignif_genes_HEK293.txt') as f:
    RNAseq_HEK293 = f.read().splitlines()
    
print("proteomics_pval:", len(proteomics_pval))
print("proteomics_adjpval:", len(proteomics_adjpval))
print("proteomics_high_rval:", len(proteomics_high_rval))
print("RNAseq_HEK293:", len(RNAseq_HEK293))

proteomics_pval: 532
proteomics_adjpval: 22
proteomics_high_rval: 302
RNAseq_HEK293: 139


In [84]:
# Open the .csv file
# GeCKO screens data
GeCKO_sp1rep = pandas.read_csv(GeCKO_path+'GeCKO_SP1_repressors.csv', sep = ',')['Gene Symbol']
GeCKO_sp1activ = pandas.read_csv(GeCKO_path+'GeCKO_SP1_activators.csv', sep = ',')['Gene Symbol']
print("GeCKO SP1 repressors:", len(GeCKO_sp1rep), "\nGeCKO SP1 activators:", len(GeCKO_sp1activ))

GeCKO_rmb3rep = pandas.read_csv(GeCKO_path+'GeCKO_SP1_repressors.csv', sep = ',')['Gene Symbol']
print("GeCKO RMB3 repressor:", len(GeCKO_rmb3rep), '\n')

# RNAseq Mice
## Cortex
RNAseq_cortex_down = pandas.read_csv(RNAseq_path2+'DESeq_in_vivo_cortex_temp_downregulated.csv', sep = ',')['gene']
RNAseq_cortex_up = pandas.read_csv(RNAseq_path2+'DESeq_in_vivo_cortex_temp_upregulated.csv', sep = ',')['gene']
print("RNAseq_cortex_down:", len(RNAseq_cortex_down), "\nRNAseq_cortex_up:", len(RNAseq_cortex_up))

## Hippocampus
RNAseq_hippoc_down = pandas.read_csv(RNAseq_path2+'DESeq_in_vivo_hippocampus_temp_downregulated.csv', sep = ',')['gene']
RNAseq_hippoc_up = pandas.read_csv(RNAseq_path2+'DESeq_in_vivo_hippocampus_temp_upregulated.csv', sep = ',')['gene']
print("RNAseq_hippoc_down:", len(RNAseq_hippoc_down), "\nRNAseq_hippoc_up:", len(RNAseq_hippoc_up))

GeCKO SP1 repressors: 495 
GeCKO SP1 activators: 61
GeCKO RMB3 repressor: 495 

RNAseq_cortex_down: 2446 
RNAseq_cortex_up: 2399
RNAseq_hippoc_down: 1726 
RNAseq_hippoc_up: 1990


# Overlap

In [76]:
# Convert lists to sets and find the intersection
print('overlapping Proteomic and GeCKO screens:\n')

datas1 = [proteomics_high_rval, proteomics_adjpval]
labels1 = ['Proteomic-HEK293 rvalue >0.6 (abs)', 'adjpvalue <0.05 Proteomic-HEK293'] 

datas2 = [GeCKO_sp1rep, GeCKO_sp1activ, GeCKO_rmb3rep]
labels2 = ['GeCKO_sp1rep', 'GeCKO_sp1activ', 'GeCKO_rmb3rep']

for i in range(len(datas1)):
    data1 = datas1[i]
    label1 = labels1[i]
    
    for y in range(len(datas2)):
        data2 = datas2[y]
        label2 = labels2[y]
        
        common_elements = set(data1).intersection(data2)
        common_elements = list(common_elements)
        print(len(common_elements), f"overlap {label1} and the {label2}:\n", common_elements, "\n")

overlapping Proteomic and GeCKO screens:

8 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the GeCKO_sp1rep:
 ['NOC3L', 'RPL13', 'DCTN3', 'RBM8A', 'MATR3', 'SMYD5', 'RPS8', 'ARFGAP1'] 

1 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the GeCKO_sp1activ:
 ['HNRNPDL'] 

8 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the GeCKO_rmb3rep:
 ['NOC3L', 'RPL13', 'DCTN3', 'RBM8A', 'MATR3', 'SMYD5', 'RPS8', 'ARFGAP1'] 

2 overlap adjpvalue <0.05 Proteomic-HEK293 and the GeCKO_sp1rep:
 ['RBM8A', 'RPL13'] 

0 overlap adjpvalue <0.05 Proteomic-HEK293 and the GeCKO_sp1activ:
 [] 

2 overlap adjpvalue <0.05 Proteomic-HEK293 and the GeCKO_rmb3rep:
 ['RBM8A', 'RPL13'] 



In [85]:
# Convert lists to sets and find the intersection
print('overlapping Proteomic and RNAseq:\n')

datas1 = [proteomics_high_rval, proteomics_adjpval]
labels1 = ['Proteomic-HEK293 rvalue >0.6 (abs)', 'adjpvalue <0.05 Proteomic-HEK293'] 

datas2 = [RNAseq_HEK293, RNAseq_hippoc_down, RNAseq_hippoc_up, RNAseq_cortex_down, RNAseq_cortex_up]
labels2 = ['RNAseq_HEK293','RNAseq_hippoc_down','RNAseq_hippoc_up','RNAseq_cortex_down','RNAseq_cortex_up']

for i in range(len(datas1)):
    data1 = datas1[i]
    label1 = labels1[i]
    
    for y in range(len(datas2)):
        data2 = datas2[y]
        label2 = labels2[y]
        
        common_elements = set(data1).intersection(data2)
        common_elements = list(common_elements)
        print(len(common_elements), f"overlap {label1} and the {label2}:\n", common_elements, "\n")

overlapping Proteomic and RNAseq:

6 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the RNAseq_HEK293:
 ['NOC3L', 'SRSF5', 'CHORDC1', 'RBM3', 'CIRBP', 'HNRNPDL'] 

0 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the RNAseq_hippoc_down:
 [] 

0 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the RNAseq_hippoc_up:
 [] 

0 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the RNAseq_cortex_down:
 [] 

0 overlap Proteomic-HEK293 rvalue >0.6 (abs) and the RNAseq_cortex_up:
 [] 

2 overlap adjpvalue <0.05 Proteomic-HEK293 and the RNAseq_HEK293:
 ['CIRBP', 'SRSF5'] 

0 overlap adjpvalue <0.05 Proteomic-HEK293 and the RNAseq_hippoc_down:
 [] 

0 overlap adjpvalue <0.05 Proteomic-HEK293 and the RNAseq_hippoc_up:
 [] 

0 overlap adjpvalue <0.05 Proteomic-HEK293 and the RNAseq_cortex_down:
 [] 

0 overlap adjpvalue <0.05 Proteomic-HEK293 and the RNAseq_cortex_up:
 [] 

